In [403]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [404]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]

In [405]:
df = pd.DataFrame()
for year in range(1990,2024):
    df_temp = pd.read_csv('game_data/gl'+str(year)+'.txt', header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df,df_temp))

In [406]:
df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df = df[df['run_diff'] != 0]

In [407]:
df_mets = df.loc[((df.team_v=='SLN') | (df.team_h=='SLN')), :]
df_mets.head(100)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,...,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total
9,19900409,0,Mon,MON,NL,1,SLN,NL,1,5,...,4,magrj001,Joe Magrane,1,NaN,Y,1990,1,1,11
18,19900410,0,Tue,MON,NL,2,SLN,NL,2,2,...,4,smitb001,Bryn Smith,1,NaN,Y,1990,2,1,6
29,19900411,0,Wed,MON,NL,3,SLN,NL,3,6,...,4,delej001,Jose DeLeon,1,NaN,Y,1990,-2,0,10
49,19900413,0,Fri,SLN,NL,4,PHI,NL,4,11,...,2,howek001,Ken Howell,1,NaN,Y,1990,-11,0,11
61,19900414,0,Sat,SLN,NL,5,PHI,NL,5,2,...,2,ruffb001,Bruce Ruffin,1,NaN,Y,1990,4,1,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,19900724,0,Tue,CHN,NL,97,SLN,NL,96,4,...,9,magrj001,Joe Magrane,1,NaN,Y,1990,5,1,13
1248,19900725,0,Wed,CHN,NL,98,SLN,NL,97,0,...,9,tudoj001,John Tudor,1,NaN,Y,1990,9,1,9
1255,19900726,1,Thu,SLN,NL,98,NYN,NL,94,1,...,8,darlr001,Ron Darling,1,NaN,Y,1990,5,1,7
1256,19900726,2,Thu,SLN,NL,99,NYN,NL,95,3,...,2,ojedb001,Bob Ojeda,1,NaN,Y,1990,-2,0,4


In [408]:
df.shape

(78356, 165)

In [409]:
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

def create_team(team, suffix, cols, stripped_cols, home_game_val):
    df_team_section = df[df[f'team_{suffix}'] == team]
    opponent = df_team_section[f'team_{suffix}'].replace({team: None})
    df_team_section = df_team_section[cols]
    df_team_section.columns = stripped_cols
    df_team_section['home_game'] = home_game_val
    df_team_section['opponent'] = opponent
    return df_team_section

def team_df(team):
    df_team_v = create_team(team, 'v', visit_cols, visit_cols_stripped, 0)
    df_team_h = create_team(team, 'h', home_cols, home_cols_stripped, 1)

    df_team = pd.concat([df_team_h, df_team_v])
    df_team.sort_values(['date', 'game_no'], inplace=True)
    
    for winsize in [162,27]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                + 2*df_team['rollsum_3B_'+suff]+ 
                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)

In [410]:
team_data_dict = {}
for team in df['team_h'].unique():
    team_data_dict[team] = team_df(team)

statistics_df = pd.DataFrame()

for index, row in df.iterrows():

    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season'] * 1000 + row['game_no_v']
    game_index_h = row['season'] * 1000 + row['game_no_h']

    for suffix in ['_162', '_27']:
        for team, team_data in [(home_team, 'h'), (visit_team, 'v')]:
            team_stats = team_data_dict[team].loc[game_index_h if team_data == 'h' else game_index_v]
            for stat in ['BATAVG', 'OBP', 'SLG', 'OBS', 'SB', 'CS', 'ERR']:
                statistics_df.loc[index, f'{stat}{suffix}_{team_data}'] = team_stats[f'rollsum_{stat}{suffix}']

In [411]:
df = pd.concat([df, statistics_df], axis=1)
df.shape

(78356, 193)

In [412]:
df.head()

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,...,SB_27_h,CS_27_h,ERR_27_h,BATAVG_27_v,OBP_27_v,SLG_27_v,OBS_27_v,SB_27_v,CS_27_v,ERR_27_v
0,19900409,0,Mon,DET,AL,1,BOS,AL,1,2,...,21.0,5.0,15.0,0.229989,0.305894,0.396843,0.702738,17.0,6.0,16.0
1,19900409,0,Mon,SEA,AL,1,CAL,AL,1,7,...,5.0,6.0,18.0,0.234375,0.307071,0.381696,0.688767,22.0,4.0,24.0
2,19900409,0,Mon,MIL,AL,1,CHA,AL,1,1,...,11.0,3.0,19.0,0.219756,0.272257,0.358491,0.630747,31.0,5.0,11.0
3,19900409,0,Mon,BAL,AL,1,KCA,AL,1,7,...,13.0,2.0,8.0,0.274678,0.344961,0.442060,0.787021,11.0,2.0,5.0
4,19900409,0,Mon,MIN,AL,1,OAK,AL,1,3,...,11.0,2.0,10.0,0.231694,0.289899,0.398907,0.688806,6.0,3.0,13.0


In [414]:
# Split the data into train, validation, and test sets
df_train = df[(df.season > 1980) & (df.season <= 2018)]
df_valid = df[(df.season >= 2019) & (df.season <= 2020)]
df_test = df[df.season >= 2021]

# Step 2: Feature Engineering
features = [
    'BATAVG_162_h', 'OBP_162_h', 'SLG_162_h', 'OBS_162_h', 'SB_162_h', 'CS_162_h', 'ERR_162_h',
    'BATAVG_162_v', 'OBP_162_v', 'SLG_162_v', 'OBS_162_v', 'SB_162_v', 'CS_162_v', 'ERR_162_v',
    'BATAVG_27_h', 'OBP_27_h', 'SLG_27_h', 'OBS_27_h', 'SB_27_h', 'CS_27_h', 'ERR_27_h',
    'BATAVG_27_v', 'OBP_27_v', 'SLG_27_v', 'OBS_27_v', 'SB_27_v', 'CS_27_v', 'ERR_27_v'
]
target = 'home_victory'

X_train = df_train.loc[:, features]
X_valid = df_valid.loc[:, features]
X_test = df_test.loc[:, features]

y_train = df_train[target].to_numpy()
y_valid = df_valid[target].to_numpy()
y_test = df_test[target].to_numpy()

# Step 3: Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 300, 500, 1000],
    'learning_rate': [0.01, 0.02, 0.1, 0.2],
    'max_depth': [2, 3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_model = XGBClassifier(random_state=42)
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5, verbose=1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

xgb_model = XGBClassifier(**best_params, random_state=42)
xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='logloss', early_stopping_rounds=50, verbose=10)

# Step 4: Cross-Validation
cv_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy: {cv_scores.mean()} ± {cv_scores.std()}")

# Step 5: Model Ensemble
logreg = LogisticRegression(random_state=42)
rf = RandomForestClassifier(random_state=42)

ensemble_model = VotingClassifier(estimators=[
    ('logreg', logreg),
    ('rf', rf),
    ('xgb', xgb_model)
], voting='soft')

ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 100, 'subsample': 0.8}
[0]	validation_0-logloss:0.69044
[10]	validation_0-logloss:0.69046
[20]	validation_0-logloss:0.69048
[30]	validation_0-logloss:0.69047


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[40]	validation_0-logloss:0.69050
[50]	validation_0-logloss:0.69051
Cross-validation accuracy: 0.5380277531739003 ± 2.9524653085344353e-05
Accuracy: 0.5196871998902456
Confusion Matrix:
[[ 461 2957]
 [ 544 3327]]
Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.13      0.21      3418
           1       0.53      0.86      0.66      3871

    accuracy                           0.52      7289
   macro avg       0.49      0.50      0.43      7289
weighted avg       0.50      0.52      0.45      7289

